In [1]:
!pip install datasets

!pip install transformers
!pip install sentencepiece

!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 4.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/webnlg

/content/drive/MyDrive/webnlg


In [13]:
import datasets, re
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from torch.nn.parallel import DataParallel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
"""
model_reasoning = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
model_reasoning = DataParallel(model_reasoning)

model_output = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
model_output = DataParallel(model_output)
"""
# to train single model:
model_output_single = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
model_output_single = DataParallel(model_output_single)

new_tokens = ['<H>', '<R>', '<T>', '[SEP]']
new_tokens_vocab = {}
new_tokens_vocab['additional_special_tokens'] = []
for idx, t in enumerate(new_tokens):
    new_tokens_vocab['additional_special_tokens'].append(t)
num_added_toks = tokenizer.add_special_tokens(new_tokens_vocab)

tokenizer.add_tokens("[MASK]")
tokenizer.mask_token = "[MASK]"
tokenizer.mask_token_id = tokenizer.convert_tokens_to_ids("[MASK]")

In [14]:
tokenizer

T5TokenizerFast(name_or_path='google/flan-t5-small', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '[MASK]', 'additional_special_tokens': ['<H>', '<R>', '<T>', '[SEP]']}, clean_up_tokenization_spaces=True)

In [13]:
class WebNLGDatasetReasoning(Dataset):
    def __init__(self, data):
        self.data = data
        self.prefix = "translate from Graph to Text: "

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # input: triples
        try:
            triples = item['modified_triple_sets']['mtriple_set']
            input_text = self.prefix
            for outer_list in triples:
                for triple in outer_list:
                    triple_txt = triple.split("|")
                    input_text += " <H> " + triple_txt[0] + " <R> " + triple_txt[1] + " <T> " + triple_txt[2] + " [SEP]"
        except (KeyError, IndexError):
            print("1 - WebNLGDatasetReasoning")
            print(item['modified_triple_sets']['mtriple_set'])
            print(item['modified_triple_sets']['mtriple_set'][0])
            print(triples)
            input_text = self.prefix
        # target: reasoning
        try:
            target_text = item['reasoning']
        except (KeyError, IndexError):
            print("2 - WebNLGDatasetReasoning")
            print(item)
            #print(item['original_triple_sets']['otriple_set'])
            target_text = ""
        #print(item)
        #print(input_text)
        # encode the inputs and targets using the tokenizer
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        #print(input_text)
        #print(target_text)
        return input_ids.squeeze(0), target_ids.squeeze(0)


In [7]:
class WebNLGDatasetOutput(Dataset):
    def __init__(self, data):
        self.data = data
        self.prefix = "translate from Graph to Text: "

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # input: reasoning
        try:
            input_text = item['reasoning']
        except (KeyError, IndexError):
            print("1 - WebNLGDatasetOutput")
            print(item['original_triple_sets']['otriple_set'])
            print(item['original_triple_sets']['otriple_set'][0])
            print(item['reasoning'])
            input_text = self.prefix
        # target: text
        try:
            target_text = item['lex']['text'][0]
        except (KeyError, IndexError):
            print("2 - WebNLGDatasetOutput")
            print(item)
            #print(item['original_triple_sets']['otriple_set'])
            target_text = ""
        #print(item)
        #print(input_text)
        # encode the inputs and targets using the tokenizer
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        #print(input_text)
        #print(target_text)
        return input_ids.squeeze(0), target_ids.squeeze(0)


In [15]:
# for single triple->text model
class WebNLGDatasetSingle(Dataset):
    def __init__(self, data):
        self.data = data
        self.prefix = "translate from Graph to Text: "

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # input: triples
        try:
            triples = item['modified_triple_sets']['mtriple_set']
            input_text = self.prefix
            for outer_list in triples:
                for triple in outer_list:
                    triple_txt = triple.split("|")
                    input_text += " <H> " + triple_txt[0] + " <R> " + triple_txt[1] + " <T> " + triple_txt[2] + " [SEP]"

        except (KeyError, IndexError):
            print("1 - WebNLGDatasetSingle")
            print(item['modified_triple_sets']['mtriple_set'])
            print(item['modified_triple_sets']['mtriple_set'][0])
            print(triples)
            input_text = self.prefix
        # target: text
        try:
            target_text = item['lex']['text'][0]
        except (KeyError, IndexError):
            print("2 - WebNLGDatasetSingle")
            print(item)
            target_text = ""
        #print(item)
        #print(input_text)
        # encode the inputs and targets using the tokenizer
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        #print(input_text)
        #print(target_text)
        return input_ids.squeeze(0), target_ids.squeeze(0)


In [16]:
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128
tokenizer.model_max_length = MAX_INPUT_LENGTH
"""
model_reasoning.module.config.max_length = MAX_TARGET_LENGTH
model_output.module.config.max_length = MAX_TARGET_LENGTH
"""
model_output_single.module.config.max_length = MAX_TARGET_LENGTH

# set up the device (GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
"""
model_reasoning.to(device)
model_output.to(device)
"""
model_output_single.to(device)

DataParallel(
  (module): T5ForConditionalGeneration(
    (shared): Embedding(32128, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=384, bias=False)
                (k): Linear(in_features=512, out_features=384, bias=False)
                (v): Linear(in_features=512, out_features=384, bias=False)
                (o): Linear(in_features=384, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 6)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=512, out_features=1024, bias=False)
                (wi_1): Linear(

In [17]:
import pickle

with open('final_dataset.pkl', 'rb') as f:
  final_dataset = pickle.load(f)

In [9]:
import random

random.Random(32).shuffle(final_dataset)

In [10]:
dataset_raw_train = final_dataset[:700]
dataset_raw_val = final_dataset[700:900]
dataset_raw_test = final_dataset[900:]

In [11]:
train_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=dataset_raw_train))
val_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=dataset_raw_val))
test_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=dataset_raw_test))

In [18]:
"""
dataset_train_reasoning = WebNLGDatasetReasoning(train_dataset)
dataset_val_reasoning = WebNLGDatasetReasoning(val_dataset)
dataset_test_reasoning = WebNLGDatasetReasoning(test_dataset)
dataset_train_output = WebNLGDatasetOutput(train_dataset)
dataset_val_output = WebNLGDatasetOutput(val_dataset)
dataset_test_output = WebNLGDatasetOutput(test_dataset)
"""
dataset_train_output_single = WebNLGDatasetSingle(train_dataset)
dataset_val_output_single = WebNLGDatasetSingle(val_dataset)
dataset_test_output_single = WebNLGDatasetSingle(test_dataset)

In [19]:
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=3, verbose=False):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss


# Adaptive pretraining

For STA, we fine-tuned the PLMs on a small amount of labeled data from the target task using a maximum likelihood estimation (MLE) objective. This involves training the model to maximize the likelihood of generating the correct output given the input graph and labeled data. This process helps to further adapt the PLM to the specific requirements of the target task and improve its performance on that task.

In [19]:
test_dataset[0]

{'category': 'Astronaut',
 'size': 5,
 'eid': 'Id72',
 'original_triple_sets': {'otriple_set': [['Elliot_See | status | "Deceased"@en',
    'Elliot_See | deathPlace | St._Louis',
    'Elliot_See | birthPlace | Dallas',
    'Elliot_See | occupation | Test_pilot',
    'Elliot_See | nationality | United_States']]},
 'modified_triple_sets': {'mtriple_set': [['Elliot_See | status | "Deceased"',
    'Elliot_See | deathPlace | St._Louis',
    'Elliot_See | birthPlace | Dallas',
    'Elliot_See | occupation | Test_pilot',
    'Elliot_See | nationality | United_States']]},
 'shape': '(X (X) (X) (X) (X) (X))',
 'shape_type': 'sibling',
 'lex': {'comment': ['good', 'good', 'good'],
  'lang': ['', '', ''],
  'lid': ['Id1', 'Id2', 'Id3'],
  'text': ['Elliot See was a US citizen, who was born in Dallas. He was a test pilot. He died in St. Louis.',
   'Elliot See who was originally born in Dallas, USA, worked as a test pilot before he died in St. Louis.',
   'Elliot See, a test pilot, has died in St 

In [ ]:
import random

pretrain_texts = []
for sample in dataset_raw_train:
    try:
        text = sample['lex']['text'][0]
        pretrain_texts.append(text)
    except (KeyError, IndexError):
        continue

tokenized_inputs = tokenizer(pretrain_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']

pretrain_data = torch.utils.data.TensorDataset(input_ids, attention_mask)

pretrain_loader = torch.utils.data.DataLoader(pretrain_data, batch_size=int(60), shuffle=True)

pretrain_optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
pretrain_criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

pretrain_epochs = 30  # Set the number of pre-training epochs
masking_prob = 0.15  # Probability of masking a token


# Prepare validation data
val_texts = []
for sample in dataset_raw_val:
    try:
        text = sample['text'][0]
        val_texts.append(text)
    except (KeyError, IndexError):
        continue

tokenized_inputs_val = tokenizer(val_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
input_ids_val = tokenized_inputs_val['input_ids']
attention_mask_val = tokenized_inputs_val['attention_mask']

val_data = TensorDataset(input_ids_val, attention_mask_val)

val_loader = DataLoader(val_data, batch_size=int(60), shuffle=True)

early_stopping = EarlyStopping(patience=3, verbose=True)

if tokenizer.mask_token is None:
    # Manually set a mask token if not already defined
    tokenizer.add_tokens("[MASK]")
    tokenizer.mask_token = "[MASK]"
    tokenizer.mask_token_id = tokenizer.convert_tokens_to_ids("[MASK]")

for epoch in range(pretrain_epochs):
    running_loss = 0.0
    for inputs, attention_mask in pretrain_loader:
        inputs = inputs.to(device)
        attention_mask = attention_mask.to(device)
        batch_size, seq_length = inputs.shape

        # Create a mask for randomly selected tokens
        mask = torch.rand(inputs.shape) < masking_prob

        # Randomly replace selected tokens with [MASK] token
        masked_inputs = inputs.clone()
        masked_inputs[mask] = tokenizer.mask_token_id

        pretrain_optimizer.zero_grad()
        outputs = model(input_ids=masked_inputs, attention_mask=attention_mask, decoder_input_ids=inputs)

        # Compute the loss only for the masked tokens
        masked_logits = outputs.logits[mask]
        masked_labels = inputs[mask]
        loss = pretrain_criterion(masked_logits.view(-1, masked_logits.size(-1)), masked_labels.view(-1))

        loss.backward()
        pretrain_optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(pretrain_data)
    print(f"Pretrain Epoch {epoch+1}/{pretrain_epochs} - loss: {epoch_loss:.4f}")

    # Validation
    model.eval()
    val_running_loss = 0.0
    for val_inputs, val_attention_mask in val_loader:
        val_inputs = val_inputs.to(device)
        val_attention_mask = val_attention_mask.to(device)
        batch_size, seq_length = val_inputs.shape

        mask = torch.rand(val_inputs.shape) < masking_prob
        masked_inputs = val_inputs.clone()
        masked_inputs[mask] = tokenizer.mask_token_id

        with torch.no_grad():
            outputs = model(input_ids=masked_inputs, attention_mask=val_attention_mask, decoder_input_ids=val_inputs)
            masked_logits = outputs.logits[mask]
            masked_labels = val_inputs[mask]
            val_loss = pretrain_criterion(masked_logits.view(-1, masked_logits.size(-1)), masked_labels.view(-1))

        val_running_loss += val_loss.item() * val_inputs.size(0)

    epoch_val_loss = val_running_loss / len(val_data)
    print(f"Val Epoch {epoch+1}/{pretrain_epochs} - loss: {epoch_val_loss:.4f}")

    early_stopping(epoch_val_loss, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break


For LMA, we first fine-tuned the PLMs on a small amount of task-specific data using a masked language modeling objective. This involves randomly masking some tokens in the input sequence and training the model to predict the masked tokens based on the context provided by the unmasked tokens. This process helps to adapt the PLM to the specific characteristics of the target task and improve its performance on that task.

# Finetuning

In [ ]:
# set up the data loader
#train_data = WebNLGDataset(dataset)
batch_size = 32
train_loader_reasoning = DataLoader(dataset_train_reasoning, batch_size=batch_size, shuffle=True)
val_loader_reasoning = DataLoader(dataset_val_reasoning, batch_size=batch_size, shuffle=True)

num_epochs = 20
optimizer = torch.optim.AdamW(model_reasoning.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
early_stopping = EarlyStopping(patience=2, verbose=True)

for epoch in range(num_epochs):
    model_reasoning.train()
    running_loss = 0.0
    for inputs, targets in train_loader_reasoning:
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model_reasoning(inputs, labels=targets)
        loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(dataset_train_reasoning)
    print(f"Train loss: {epoch_loss:.4f}")

    model_reasoning.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for val_inputs, val_targets in val_loader_reasoning:
            val_inputs = val_inputs.to(device)
            val_targets = val_targets.to(device)
            val_outputs = model_reasoning(val_inputs, labels=val_targets)
            val_loss = criterion(val_outputs.logits.view(-1, val_outputs.logits.size(-1)), val_targets.view(-1))
            running_val_loss += val_loss.item() * val_inputs.size(0)
    epoch_val_loss = running_val_loss / len(dataset_val_reasoning)
    print(f"Val loss: {epoch_val_loss:.4f}")

    early_stopping(epoch_val_loss, model_reasoning)

    if early_stopping.early_stop:
        print("Early stopping")
        break

In [ ]:
# set up the data loader
batch_size = 32
train_loader_output = DataLoader(dataset_train_output, batch_size=batch_size, shuffle=True)
val_loader_output = DataLoader(dataset_val_output, batch_size=batch_size, shuffle=True)

num_epochs = 20
optimizer = torch.optim.AdamW(model_output.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
early_stopping = EarlyStopping(patience=2, verbose=True)

for epoch in range(num_epochs):
    model_output.train()
    running_loss = 0.0
    for inputs, targets in train_loader_output:
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model_output(inputs, labels=targets)
        loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(dataset_train_output)
    print(f"Epoch {epoch+1}/{num_epochs} - loss: {epoch_loss:.4f}")

    model_output.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for val_inputs, val_targets in val_loader_output:
            val_inputs = val_inputs.to(device)
            val_targets = val_targets.to(device)
            val_outputs = model_output(val_inputs, labels=val_targets)
            val_loss = criterion(val_outputs.logits.view(-1, val_outputs.logits.size(-1)), val_targets.view(-1))
            running_val_loss += val_loss.item() * val_inputs.size(0)
    epoch_val_loss = running_val_loss / len(dataset_val_output)
    print(f"Val loss: {epoch_val_loss:.4f}")

    early_stopping(epoch_val_loss, model_output)

    if early_stopping.early_stop:
        print("Early stopping")
        break

In [20]:
# set up the data loader
#train_data = WebNLGDataset(dataset)
batch_size = 32
train_loader_output_single = DataLoader(dataset_train_output_single, batch_size=batch_size, shuffle=True)
val_loader_output_single = DataLoader(dataset_val_output_single, batch_size=batch_size, shuffle=True)

num_epochs = 20
optimizer = torch.optim.AdamW(model_output_single.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
early_stopping = EarlyStopping(patience=2, verbose=True)

for epoch in range(num_epochs):
    model_output_single.train()
    running_loss = 0.0
    for inputs, targets in train_loader_output_single:
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model_output_single(inputs, labels=targets)
        loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(dataset_train_output_single)
    print(f"Train loss: {epoch_loss:.4f}")

    model_output_single.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for val_inputs, val_targets in val_loader_output_single:
            val_inputs = val_inputs.to(device)
            val_targets = val_targets.to(device)
            val_outputs = model_output_single(val_inputs, labels=val_targets)
            val_loss = criterion(val_outputs.logits.view(-1, val_outputs.logits.size(-1)), val_targets.view(-1))
            running_val_loss += val_loss.item() * val_inputs.size(0)
    epoch_val_loss = running_val_loss / len(dataset_val_output_single)
    print(f"Val loss: {epoch_val_loss:.4f}")

    early_stopping(epoch_val_loss, model_output_single)

    if early_stopping.early_stop:
        print("Early stopping")
        break

Train loss: 1.7807
Val loss: 1.1878
Validation loss decreased (inf --> 1.187808).  Saving model ...
Train loss: 1.4565
Val loss: 1.1218
Validation loss decreased (1.187808 --> 1.121753).  Saving model ...
Train loss: 1.3567
Val loss: 1.0777
Validation loss decreased (1.121753 --> 1.077666).  Saving model ...
Train loss: 1.2744
Val loss: 1.0549
Validation loss decreased (1.077666 --> 1.054892).  Saving model ...
Train loss: 1.2104
Val loss: 1.0251
Validation loss decreased (1.054892 --> 1.025057).  Saving model ...
Train loss: 1.1589
Val loss: 1.0057
Validation loss decreased (1.025057 --> 1.005739).  Saving model ...
Train loss: 1.1089
Val loss: 0.9924
Validation loss decreased (1.005739 --> 0.992402).  Saving model ...
Train loss: 1.0691
Val loss: 0.9845
Validation loss decreased (0.992402 --> 0.984536).  Saving model ...
Train loss: 1.0378
Val loss: 0.9728
Validation loss decreased (0.984536 --> 0.972793).  Saving model ...
Train loss: 0.9971
Val loss: 0.9621
Validation loss decrease

In [ ]:
model.load_state_dict(torch.load('/kaggle/working/checkpoint.pt'))

In [21]:
# Save the entire model
"""
torch.save(model_reasoning, 'model_T5_flan_small_2020_reasoning_v2_es')
print("Model saved successfully.")
torch.save(model_output, 'model_T5_flan_small_2020_output_v2_es')
print("Model saved successfully.")
"""
torch.save(model_output_single, 'single1/models/model_T5_flan_small_2020_output_single_v1_es')
print("Model saved successfully.")

Model saved successfully.


In [45]:
# Load the model

model_reasoning = torch.load('version1/models/model_T5_flan_small_2020_reasoning_v2_es')
model_output = torch.load('version1/models/model_T5_flan_small_2020_output_v2_es')

# model_output_single = torch.load('model_T5_flan_small_2020_output_v2_es')

# Inference

## are we accounting for the multiple texts targets in the bleu? it doesn't look like it

In [22]:
batch_size=32

In [29]:
# single model
from sacrebleu import corpus_bleu
from random import sample
from tqdm import tqdm

# test_loader_output_single = DataLoader(dataset_test_output_single, batch_size=batch_size, shuffle=False)

# to test on the whole official test set:
test_dataset_tmp = load_dataset('web_nlg', 'release_v3.0_en')['test']
test_dataset_tmp = [sample for sample in test_dataset_tmp if sample['lex']['text']] # filter out samples with empty targets
test_dataset_webnlg_tmp = WebNLGDatasetSingle(test_dataset_tmp)
test_loader_output_single = DataLoader(test_dataset_webnlg_tmp, batch_size=batch_size, shuffle=False)

# switch model to evaluation mode
model_output_single.eval()

# generate predictions for the validation dataset
predictions_output_single = []
references_output_single = []
multiple_references_output_single = []
with torch.no_grad():
    for idx, (inputs, targets) in enumerate(tqdm(test_loader_output_single, desc='Validation Progress', leave=False)):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model_output_single.module.generate(inputs, max_length=MAX_TARGET_LENGTH, num_beams=4)
        # convert token IDs to strings
        predicted_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(targets, skip_special_tokens=True)
        # append predicted and target texts for BLEU evaluation
        predictions_output_single.extend(predicted_texts)
        multiple_references_output_single.extend([np.array(test_dataset)[i]["lex"]["text"] for i in range(idx*batch_size,min((idx+1)*batch_size, len(test_dataset)))])
        references_output_single.extend([np.array(test_dataset)[i]["lex"]["text"][0] for i in range(idx*batch_size,min((idx+1)*batch_size, len(test_dataset)))])

Generating train split:   0%|          | 0/13211 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1667 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5713 [00:00<?, ? examples/s]

Dataset web_nlg downloaded and prepared to /root/.cache/huggingface/datasets/web_nlg/release_v3.0_en/0.0.0/28ffb892f7f42450dd9558684aa43bcaf44b1b3bf0d77cb8d73534646af88dda. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [33]:
references_output_single = []
multiple_references_output_single = []
for i in range(3934):
    references_output_single.append(test_dataset_tmp[i]["lex"]["text"][0])
    multiple_references_output_single.append(test_dataset_tmp[i]["lex"]["text"])

In [36]:
from sacrebleu import corpus_bleu
from random import sample
from tqdm import tqdm

test_loader_reasoning = DataLoader(dataset_test_reasoning, batch_size=batch_size, shuffle=False)
# also we use the test_dataset to get the multiple references
"""
# load the WebNLG validation dataset
validation_dataset = load_dataset('web_nlg', 'release_v3.0_en')['test']
validation_dataset = [sample for sample in validation_dataset if sample['lex']['text']] # filter out samples with empty targets
validation_dataset = validation_dataset[:5]
# Select a subset of the validation dataset
#subset_size = 10  # Choose the desired subset size
#validation_subset = sample(list(validation_dataset), subset_size)
validation_data = WebNLGDataset(validation_dataset)

# set up the validation data loader
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)
"""
# switch model to evaluation mode
model_reasoning.eval()

# generate predictions for the validation dataset
predictions_reasoning = []
references_reasoning = []

with torch.no_grad():
    for idx, (inputs, targets) in enumerate(tqdm(test_loader_reasoning, desc='Validation Progress', leave=False)):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model_reasoning.module.generate(inputs, max_length=MAX_TARGET_LENGTH, num_beams=4)
        # convert token IDs to strings
        predicted_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(targets, skip_special_tokens=True)
        # append predicted and target texts for BLEU evaluation
        predictions_reasoning.extend(predicted_texts)
        references_reasoning.extend([np.array(test_dataset)[i]["reasoning"] for i in range(idx*batch_size,min((idx+1)*batch_size, len(test_dataset)))])

In [28]:
test_loader_output = DataLoader(dataset_test_output, batch_size=batch_size, shuffle=False)
# also we use the test_dataset to get the multiple references
"""
# load the WebNLG validation dataset
validation_dataset = load_dataset('web_nlg', 'release_v3.0_en')['test']
validation_dataset = [sample for sample in validation_dataset if sample['lex']['text']] # filter out samples with empty targets
validation_dataset = validation_dataset[:5]
# Select a subset of the validation dataset
#subset_size = 10  # Choose the desired subset size
#validation_subset = sample(list(validation_dataset), subset_size)
validation_data = WebNLGDataset(validation_dataset)

# set up the validation data loader
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)
"""
# switch model to evaluation mode
model_output.eval()

# generate predictions for the validation dataset
predictions_output = []
references_output = []
multiple_references_output = []
with torch.no_grad():
    for idx, (inputs, targets) in enumerate(tqdm(test_loader_output, desc='Validation Progress', leave=False)):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model_output.module.generate(inputs, max_length=MAX_TARGET_LENGTH, num_beams=4)
        # convert token IDs to strings
        predicted_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(targets, skip_special_tokens=True)
        # append predicted and target texts for BLEU evaluation
        predictions_output.extend(predicted_texts)
        multiple_references_output.extend([np.array(test_dataset)[i]["lex"]["text"] for i in range(idx*batch_size,min((idx+1)*batch_size, len(test_dataset)))])
        references_output.extend([np.array(test_dataset)[i]["lex"]["text"][0] for i in range(idx*batch_size,min((idx+1)*batch_size, len(test_dataset)))])

In [46]:
# test_loader_reasoning = DataLoader(dataset_test_reasoning, batch_size=batch_size, shuffle=False)
# test_loader_output = DataLoader(dataset_test_output, batch_size=batch_size, shuffle=False)

# to test with official test set
test_dataset_tmp = load_dataset('web_nlg', 'release_v3.0_en')['test']
test_dataset_tmp = [sample for sample in test_dataset_tmp if sample['lex']['text']] # filter out samples with empty targets
test_dataset_webnlg_tmp = WebNLGDatasetSingle(test_dataset_tmp)
test_loader_reasoning = DataLoader(test_dataset_webnlg_tmp, batch_size=batch_size, shuffle=False)
model_reasoning = model_output_single

model_reasoning.eval()
model_output.eval()

# generate predictions for the validation dataset
predictions_reasoning = []
predictions_output_pipeline = []

# Use tqdm for progress bar
with torch.no_grad():
    for idx, (inputs, targets) in enumerate(tqdm(test_loader_reasoning, desc='Validation Progress', leave=False)):
        inputs = inputs.to(device)
        targets = targets.to(device)

        # pass inputs through the reasoning model
        outputs_reasoning = model_reasoning.module.generate(inputs, max_length=MAX_TARGET_LENGTH, num_beams=4)

        # convert token IDs to strings
        predicted_texts_reasoning = tokenizer.batch_decode(outputs_reasoning, skip_special_tokens=True)

        # append predicted and target texts for BLEU evaluation
        predictions_reasoning.extend(predicted_texts_reasoning)

        # feed the reasoning model's outputs into the output model
        outputs_output = model_output.module.generate(outputs_reasoning, max_length=MAX_TARGET_LENGTH, num_beams=4)

        # convert token IDs to strings
        predicted_texts_output = tokenizer.batch_decode(outputs_output, skip_special_tokens=True)
        target_texts_output = tokenizer.batch_decode(targets, skip_special_tokens=True)

        # append predicted and target texts for BLEU evaluation
        predictions_output_pipeline.extend(predicted_texts_output)

  0%|          | 0/3 [00:00<?, ?it/s]

In [61]:
assert len(predictions_output_pipeline) == len(references_output_single)
i=20
print(test_dataset_tmp[i]['modified_triple_sets']['mtriple_set'])
print('----')
print("pipeline model prediction: ", predictions_output_pipeline[i])
print('----')
print("single model prediction: ", preds[i])
print('----')
print("single model reference: ", references_output_single[i])
print('----')

[['Bootleg_Series_Volume_1:_The_Quine_Tapes | artist | The_Velvet_Underground', 'Bootleg_Series_Volume_1:_The_Quine_Tapes | producer | The_Velvet_Underground', 'Bootleg_Series_Volume_1:_The_Quine_Tapes | runtime | 230.05', 'Bootleg_Series_Volume_1:_The_Quine_Tapes | recordedIn | United_States', 'Bootleg_Series_Volume_1:_The_Quine_Tapes | recordedIn | St._Louis,_Missouri', 'Bootleg_Series_Volume_1:_The_Quine_Tapes | genre | Rock_music', 'Bootleg_Series_Volume_1:_The_Quine_Tapes | recordLabel | Polydor_Records']]
----
pipeline model prediction:  The Velvet Underground is the producer of the Bootleg Series Volume 1: The Beast Tapes which has a run time of 230.05 and a run time of 230.05. It is the producer of the Bootleg Series Volume 1: The Beast Tapes which has a run time of 230.05 and a run time of 230.05.
----
single model prediction:  The Velvet Underground is the producer of the Bootleg Series Volume 1: The Beast Tapes which has a run time of 230.05. It is produced by The Velvet Und

In [55]:
with open("/content/drive/MyDrive/webnlg/single1/outputs2/predictions", "r") as f:
  preds = f.readlines()

In [56]:
preds[0]

'Agremiaço Sportiva Arapiraquense play in the Campeonato Brasileiro Serie C league in Brazil. They play in the same league as the Estadio Municipal College da Mata Fonseca in Arapiraca.\n'

In [63]:
assert len(predictions_reasoning) == len(references_reasoning) == len(predictions_output) == len(predictions_output_pipeline) == len(references_output) == len(multiple_references_output)
i=2
print(test_dataset[i]['modified_triple_sets']['mtriple_set'])
print('----')
print("reasoning prediction: ", predictions_reasoning[i])
print('----')
print("reasoning reference: ", references_reasoning[i])
print('----')
print("output prediction: ", predictions_output[i])
print('----')
print("output pipeline prediction: ", predictions_output_pipeline[i])
print('----')
print("output reference: ", references_output[i])
print('----')
print("output multiple reference: ", multiple_references_output[i])

[['Allen_Forrest | genre | Rhythm_and_blues', 'Allen_Forrest | birthYear | 1981', 'Allen_Forrest | birthPlace | "Fort Campbell, KY, raised in Dothan, AL"', 'Allen_Forrest | background | "solo_singer"', 'Allen_Forrest | birthPlace | Fort_Campbell']]
----
reasoning prediction:  Allen Forrest's genre is rhythm and blues. Allen Forrest was born in 1981. Allen Forrest was born in Fort Campbell, KY, raised in Dothan, AL. Allen Forrest has a background as a solo singer. Allen Forrest was born in Fort Campbell, KY.
----
reasoning reference:  Allen Forrest's genre is Rhythm and blues. Allen Forrest was born in 1981. Allen Forrest was born in Fort Campbell, KY and raised in Dothan, AL. Allen Forrest has a background as a solo singer.
----
output prediction:  Allen Forrest was born in 1981 in Fort Campbell, KY and raised in Dothan, AL. He performs rhythm and blues music and has a background as a solo singer.
----
output pipeline prediction:  Allen Forrest was born in 1981 in Fort Campbell, KY, ra

In [27]:
# Writing predictions to a .txt file
import os
def write_outputs(predictions_reasoning, references_reasoning, predictions_output, references_output, multiple_references_output, predictions_output_pipeline, folder):
    if not os.path.exists(folder):
        os.mkdir(folder)
    models = ['reasoning', 'output', 'output_pipeline']
    for f in models:
        inner = os.path.join(folder, f)
        if not os.path.exists(inner):
            os.mkdir(inner)
    with open(os.path.join(os.path.join(folder, "reasoning"), "predictions"), "w") as f:
        for prediction in predictions_reasoning:
            f.write(prediction + "\n")
    with open(os.path.join(os.path.join(folder, "output"), "predictions"), "w") as f:
        for prediction in predictions_output:
            f.write(prediction + "\n")
    with open(os.path.join(os.path.join(folder, "output_pipeline"), "predictions"), "w") as f:
        for prediction in predictions_output_pipeline:
            f.write(prediction + "\n")

    # now write the references files to each folder
    write_references([[ref] for ref in references_reasoning], os.path.join(folder, "reasoning"))
    write_references(multiple_references_output, os.path.join(folder, "output"))
    write_references(multiple_references_output, os.path.join(folder, "output_pipeline"))

def write_references(multiple_references, folder):
    ref_folder = os.path.join(folder, "references")
    if not os.path.exists(ref_folder):
        os.mkdir(ref_folder)

    # Getting the maximum length of the sublists in multiple_references
    max_length = max(len(sublist) for sublist in multiple_references)

    # Writing multiple_references to separate .txt files
    for i in range(max_length):
        with open(os.path.join(ref_folder, f"references{i}"), "w") as f:
            for ref_list in multiple_references:
                # Writing the ith element if it exists, otherwise an empty line
                if i < len(ref_list):
                    f.write(ref_list[i] + "\n")
                else:
                    f.write("\n")

In [51]:
folder = "version1/whole_Set"
if not os.path.exists(folder):
    os.mkdir(folder)
with open(os.path.join(folder, "predictions"), "w") as f:
    for prediction in predictions_output_pipeline:
        f.write(prediction + "\n")
write_references(multiple_references_output_single, folder)


In [73]:
! rm -rf /content/drive/MyDrive/webnlg/version1/

In [ ]:
write_outputs(predictions_reasoning, references_reasoning, predictions_output, references_output, multiple_references_output, '/content/drive/MyDrive/webnlg/version1/whole_set/')

In [ ]:
# Read predictions from a .txt file
with open("predictions_reasoning_version2_es", "r") as f:
    predictions_reasoning = f.readlines()
with open("predictions_output_version2_es", "w") as f:
    for prediction in predictions_output:
        f.write(prediction + "\n")
with open("predictions_output_pipeline_version2_es", "r") as f:
    predictions_output_pipeline = f.readlines()

In [31]:
# calculate BLEU scores
#bleu = corpus_bleu(predictions, [references])
"""
multiple_references = []
for i in range(len(validation_dataset)):
    multiple_references.append(validation_dataset[i]['lex']['text'])
"""
bleu = corpus_bleu(predictions_reasoning, references_reasoning)
# bleu_multiple = corpus_bleu(predictions, multiple_references)

print(f"BLEU score reasoning: {bleu.score}")
# print(f"BLEU score with multiple references: {bleu_multiple.score}")

bleu = corpus_bleu(predictions_output, references_output)
bleu_multiple = corpus_bleu(predictions_output, multiple_references_output)

print(f"BLEU score output: {bleu.score}")
print(f"BLEU score with multiple references: {bleu_multiple.score}")

BLEU score reasoning: 0.02135099363252779
BLEU score output: 0.025183204076820096
BLEU score with multiple references: 77.68917956332257


In [71]:
# Getting the maximum length of the sublists in multiple_references
max_length = max(len(sublist) for sublist in multiple_references_output)

# Writing multiple_references to separate .txt files
for i in range(max_length):
    with open(f"references{i}", "w") as f:
        for ref_list in multiple_references_output:
            # Writing the ith element if it exists, otherwise an empty line
            if i < len(ref_list):
                f.write(ref_list[i] + "\n")
            else:
                f.write("\n")

In [49]:
from datasets import load_metric

metric = load_metric('sacrebleu')

# First, determine the maximum length of sublists
max_len = max(len(refs) for refs in multiple_references_output)

# Then pad all sublists to that length
padded_references = []
for ref in multiple_references_output:
  for i in range(max_len-len(ref)):
    ref.append("")
  padded_references.append(ref)
# padded_references = [refs * (max_len // len(refs)) + refs[:max_len % len(refs)] for refs in multiple_references_output]

# Now 'padded_references' is a list of lists, where each sublist has the same length.
# We can now compute the SacreBLEU score.

# Note the change in the compute line
score_reasoning = metric.compute(predictions=predictions_reasoning, references = padded_references)

print(f"SacreBLEU score: {score['score']}")

<ipython-input-49-c2e077e1ff4d>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('sacrebleu')


SacreBLEU score: 46.527816771672605


In [50]:
from sacrebleu import corpus_chrf
# Calculate CHR F++ scores
chrf = corpus_chrf(predictions_output, [references_output])
chrf_multiple = corpus_chrf(predictions_output, multiple_references_output)
print(f"CHR F++ score: {chrf.score}")
print(f"CHR F++ score with multiple references: {chrf_multiple.score}")

CHR F++ score: 64.69488887291493
CHR F++ score with multiple references: 59.48710404678748


In [34]:
from sacrebleu import corpus_chrf
# Calculate CHR F++ scores
chrf = corpus_chrf(predictions_output, [references_output])
chrf_multiple = corpus_chrf(predictions_output, padded_references)
print(f"CHR F++ score: {chrf.score}")
print(f"CHR F++ score with multiple references: {chrf_multiple.score}")

CHR F++ score: 62.58863281228343
CHR F++ score with multiple references: 55.839157211548326


In [25]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00


In [49]:
from datasets import load_metric
import numpy as np


metric = load_metric('bertscore')
"""
assert len(predictions_reasoning) == len(references_reasoning), "The number of predictions and references should be the same."
assert len(predictions_output) == len(references_output), "The number of predictions and references should be the same."
assert len(predictions_output_pipeline) == len(references_output), "The number of predictions and references should be the same."

# Compute the score
score_reasoning = metric.compute(predictions=predictions_reasoning, references=references_reasoning, lang='en')
score_output = metric.compute(predictions=predictions_output, references=references_output, lang='en')
score_output_pipeline = metric.compute(predictions=predictions_output_pipeline, references=references_output, lang='en')

print(f"BERTScore Reasoning: {np.mean(score_reasoning['precision'])}")
print(f"BERTScore Output: {np.mean(score_output['precision'])}")
print(f"BERTScore Output Pipeline: {np.mean(score_output_pipeline['precision'])}")
"""
assert len(predictions_output_single) == len(references_output_single), "The number of predictions and references should be the same."
score_output_single = metric.compute(predictions=predictions_output_pipeline, references=references_output_single, lang='en')
print(f"BERTScore Output Single: {np.mean(score_output_single['precision'])}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore Output Single: 0.8967975244554821


In [ ]:
from transformers import T5Tokenizer
from datasets import load_dataset

# Define the tokenizer
#tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Load the WebNLG dataset
dataset = load_dataset('web_nlg', 'release_v3.0_en')['test']
dataset = [sample for sample in dataset if sample['lex']['text']]

# Create an instance of WebNLGDataset
webnlg_dataset = WebNLGDataset(dataset)

# Define the index of the example you want to test
example_index = 70

# Get the input and target texts for the example at the specified index
input_text, target_text = webnlg_dataset[example_index]

# Decode the input and target texts using the tokenizer
decoded_input_text = tokenizer.decode(input_text, skip_special_tokens=True)
decoded_target_text = tokenizer.decode(target_text, skip_special_tokens=True)

# Print the preprocessed input and target texts
print("Input Text:", decoded_input_text)
print("Target Text:", decoded_target_text)


## seeing how many empty targets there are in the testing set

In [ ]:
dataset = load_dataset('web_nlg', 'release_v3.0_en')['test']
count_empty_text = 0
for sample in dataset:
    if not sample['lex']['text']:
        count_empty_text += 1

print(f"Number of samples with empty 'lex' 'text' field: {count_empty_text}")


In [ ]:
total_samples = len(dataset)
print(f"Total number of samples in the test dataset: {total_samples}")